In [22]:
# -----------------------------
# Trader vs Sentiment
# -----------------------------
#
# It will:
#  - install required packages,
#  - load /mnt/data/historical_data.csv and /mnt/data/fear_greed_index.csv (fallback to cwd),
#  - analyze, plot, create HTML report with embedded images,
#  - convert HTML->PDF (WeasyPrint preferred, ReportLab fallback),
#  - save outputs and a Streamlit dashboard script.
# -----------------------------

# -----------------------------
# 0) Install system deps & pip packages (Colab friendly)
# -----------------------------
import sys, os
print("Installing required packages (this may take a minute)...", flush=True)
# apt-get deps needed by weasyprint (Colab)
os.system("apt-get update -qq > /dev/null")
os.system("apt-get install -y -qq libpangocairo-1.0-0 libpango1.0-0 libcairo2 libgdk-pixbuf2.0-0 libffi-dev shared-mime-info > /dev/null || true")
# pip installs
os.system("pip install --quiet pandas numpy matplotlib seaborn plotly weasyprint pillow reportlab streamlit nbconvert jinja2 scikit-learn")

# -----------------------------
# 1) Imports
# -----------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from io import BytesIO
import base64
from datetime import datetime
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from PIL import Image
import warnings
warnings.filterwarnings("ignore")
sns.set(style='whitegrid')

# Try importing weasyprint (used for HTML -> PDF)
use_weasy = True
try:
    from weasyprint import HTML
except Exception as e:
    print("WeasyPrint import failed (will attempt fallback). Error:", e)
    use_weasy = False

# -----------------------------
# 2) Paths and load CSVs
# -----------------------------
out_dir = '/content/trader_sentiment_results'
os.makedirs(out_dir, exist_ok=True)

# Candidate paths (the environment indicated the files at /mnt/data)
path1 = '/content/historical_data.csv'
path2 = '/content/fear_greed_index.csv'
fallback1 = 'historical_data.csv'
fallback2 = 'fear_greed_index.csv'

def load_csv_try(path_a, path_b):
    if os.path.exists(path_a) and os.path.exists(path_b):
        print(f"Loading {path_a} and {path_b}")
        return pd.read_csv(path_a), pd.read_csv(path_b)
    elif os.path.exists(fallback1) and os.path.exists(fallback2):
        print(f"Loading {fallback1} and {fallback2} from working dir")
        return pd.read_csv(fallback1), pd.read_csv(fallback2)
    else:
        raise FileNotFoundError(f"Could not find datasets at {path_a}/{path_b} or {fallback1}/{fallback2}. Please upload them to Colab.")

trades, sentiment = load_csv_try(path1, path2)
print("Trades shape:", trades.shape)
print("Sentiment shape:", sentiment.shape)

# -----------------------------
# 3) Basic cleaning & datetime parsing
# -----------------------------
# Normalize column names
trades.columns = trades.columns.str.strip().str.lower().str.replace(' ', '_')
sentiment.columns = sentiment.columns.str.strip().str.lower().str.replace(' ', '_')

# Identify time column in trades
time_col_candidates = ['time', 'timestamp', 'datetime', 'date']
trades['time'] = None
for c in time_col_candidates:
    if c in trades.columns:
        try:
            trades['time'] = pd.to_datetime(trades[c], errors='coerce')
            print(f"Parsed trades time from column: {c}")
            break
        except Exception:
            pass
# If still none, try numeric epoch -> datetime
if trades['time'].isna().all() and 'time' in trades.columns:
    trades['time'] = pd.to_datetime(trades['time'], unit='s', errors='coerce') # Added unit='s' for potential epoch conversion

# Sentiment: identify date and classification
date_col = None
for c in ['date', 'time', 'timestamp', 'datetime']:
    if c in sentiment.columns:
        date_col = c
        break
if date_col is None:
    raise ValueError("No date column found in sentiment CSV (expected 'date'/'time'/'timestamp').")

sentiment['date_parsed'] = pd.to_datetime(sentiment[date_col], errors='coerce').dt.date

# Identify classification column
possible_class_cols = [c for c in sentiment.columns if 'class' in c or 'fear' in c or 'greed' in c or 'index' in c]
if 'classification' in sentiment.columns:
    sentiment['classification'] = sentiment['classification']
elif len(possible_class_cols) > 0:
    # choose first plausible (prefer exact names)
    chosen = possible_class_cols[0]
    sentiment['classification'] = sentiment[chosen]
else:
    # fallback: if second column looks like labels
    cols = list(sentiment.columns)
    if len(cols) >= 2:
        sentiment['classification'] = sentiment[cols[1]]
    else:
        raise ValueError("Could not find a classification column in sentiment CSV.")

# Normalize classification strings -> Fear/Greed/Unknown
sentiment['classification'] = sentiment['classification'].astype(str).str.strip().str.title()
sentiment = sentiment[['date_parsed','classification']].rename(columns={'date_parsed':'date'})

# Create trade_date column in trades (date only)
trades['trade_date'] = trades['time'].dt.date

# Drop trades with missing time (or keep with Unknown later)
missing_times = trades['time'].isna().sum()
if missing_times > 0:
    print(f"Warning: {missing_times} trade rows have missing/invalid timestamps. They will be dropped.")
    trades = trades.dropna(subset=['time']).copy()

# -----------------------------
# 4) Merge datasets by date
# -----------------------------
merged = trades.merge(sentiment, left_on='trade_date', right_on='date', how='left')
merged['classification'] = merged['classification'].fillna('Unknown')

print("Merged shape:", merged.shape)
print("Sentiment distribution in merged:")
print(merged['classification'].value_counts())

# -----------------------------
# 5) Feature engineering
# -----------------------------
# Closed PnL column: try several variants
pnl_col = None
for c in ['closedpnl', 'closed_pnl', 'closedPnL', 'closedPnL']:
    if c in merged.columns:
        pnl_col = c
        break
# If no closed PnL, try 'pnl' or 'profit'
if pnl_col is None:
    for c in ['pnl','profit','realized_pnl']:
        if c in merged.columns:
            pnl_col = c
            break
if pnl_col is None:
    print("No obvious closed PnL column found; creating 'pnl' = 0 for all (please update mapping).")
    merged['pnl'] = 0.0
else:
    merged['pnl'] = pd.to_numeric(merged[pnl_col], errors='coerce').fillna(0.0)

# size & leverage
# Check if 'size' column exists before processing
if 'size' in merged.columns:
    merged['size'] = pd.to_numeric(merged['size'], errors='coerce').fillna(0.0)
else:
    merged['size'] = 0.0 # Create column with default 0 if not found

# Check if 'leverage' column exists before processing
if 'leverage' in merged.columns:
    merged['leverage'] = pd.to_numeric(merged['leverage'], errors='coerce').fillna(0.0)
else:
    merged['leverage'] = 0.0 # Create column with default 0 if not found


# side normalization
if 'side' in merged.columns:
    merged['side'] = merged['side'].astype(str).str.lower()
else:
    merged['side'] = ''

# win indicator
merged['win'] = (merged['pnl'] > 0).astype(int)

# leverage buckets
merged['lev_bucket'] = pd.cut(merged['leverage'].replace(0, np.nan),
                             bins=[0,1,3,5,10,20,100,1e9],
                             labels=['0-1','1-3','3-5','5-10','10-20','20-100','100+'],
                             include_lowest=True)

# per-account aggregates
if 'account' in merged.columns:
    acct_agg = merged.groupby('account').agg(
        trades_count=('pnl','count'),
        total_pnl=('pnl','sum'),
        mean_pnl=('pnl','mean'),
        win_rate=('win','mean'),
        avg_leverage=('leverage','mean')
    ).reset_index()
else:
    acct_agg = pd.DataFrame()

# per-sentiment
sent_agg = merged.groupby('classification').agg(
    trades_count=('pnl','count'),
    total_pnl=('pnl','sum'),
    mean_pnl=('pnl','mean'),
    win_rate=('win','mean'),
    avg_leverage=('leverage','mean')
).reset_index()

# -----------------------------
# 6) Exploratory plots (matplotlib/seaborn) -> save PNG files & also create base64 for embedding
# -----------------------------
def save_fig_and_b64(fig, filename, outdir=out_dir, dpi=150):
    path = os.path.join(outdir, filename)
    fig.savefig(path, bbox_inches='tight', dpi=dpi)
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', dpi=dpi)
    buf.seek(0)
    b64 = base64.b64encode(buf.read()).decode('utf-8')
    plt.close(fig)
    return path, b64

# a) Boxplot: PnL by sentiment (exclude Unknown for clarity)
fig1 = plt.figure(figsize=(10,6))
ax1 = sns.boxplot(x='classification', y='pnl', data=merged[merged['classification']!='Unknown'].replace([np.inf, -np.inf], np.nan).dropna(subset=['pnl']))
ax1.set_title('PnL distribution by Market Sentiment')
ax1.set_xlabel('Sentiment')
ax1.set_ylabel('PnL')
path1, b64_1 = save_fig_and_b64(fig1, 'pnl_by_sentiment.png')

# b) Win rate by sentiment (bar)
wr = merged[merged['classification']!='Unknown'].groupby('classification').agg(win_rate=('win','mean')).reset_index()
fig2 = plt.figure(figsize=(8,5))
ax2 = sns.barplot(x='classification', y='win_rate', data=wr)
ax2.set_title('Win Rate by Sentiment')
ax2.set_ylabel('Win Rate')
path2, b64_2 = save_fig_and_b64(fig2, 'win_rate_by_sentiment.png')

# c) Mean PnL by leverage bucket and sentiment
pivot = merged[merged['classification']!='Unknown'].groupby(['lev_bucket','classification']).agg(mean_pnl=('pnl','mean')).reset_index()
fig3 = plt.figure(figsize=(12,6))
ax3 = sns.pointplot(x='lev_bucket', y='mean_pnl', hue='classification', data=pivot, dodge=True)
ax3.set_title('Mean PnL by Leverage Bucket and Sentiment')
ax3.set_xlabel('Leverage Bucket')
ax3.set_ylabel('Mean PnL')
path3, b64_3 = save_fig_and_b64(fig3, 'pnl_by_leverage_sentiment.png')

# d) Time series: rolling 7-day total pnl
daily = merged.groupby('trade_date').agg(total_pnl=('pnl','sum'), trades=('pnl','count')).reset_index()
daily['trade_date'] = pd.to_datetime(daily['trade_date'])
daily = daily.sort_values('trade_date')
daily['rolling_pnl_7d'] = daily['total_pnl'].rolling(7, min_periods=1).mean()
fig4 = plt.figure(figsize=(12,5))
plt.plot(daily['trade_date'], daily['rolling_pnl_7d'], label='7-day rolling total PnL')
plt.title('7-day Rolling Total PnL across all traders')
plt.xlabel('Date')
plt.ylabel('Total PnL (7d rolling)')
plt.legend()
path4, b64_4 = save_fig_and_b64(fig4, 'rolling_pnl.png')

# Also create small interactive Plotly HTML (optional saved)
try:
    fig_plotly = px.box(merged[merged['classification']!='Unknown'], x='classification', y='pnl', title='PnL by Sentiment (interactive)')
    plotly_html_path = os.path.join(out_dir, 'plotly_pnl_by_sentiment.html')
    fig_plotly.write_html(plotly_html_path)
except Exception as e:
    print("Failed to produce plotly interactive HTML:", e)
    plotly_html_path = None

# -----------------------------
# 7) Statistical tests & simple predictive model
# -----------------------------
# T-test PnL Fear vs Greed (only if both exist)
fear_pnl = merged[merged['classification']=='Fear']['pnl'].dropna()
greed_pnl = merged[merged['classification']=='Greed']['pnl'].dropna()
ttest_res = None
if len(fear_pnl) > 1 and len(greed_pnl) > 1:
    ttest_res = stats.ttest_ind(fear_pnl, greed_pnl, equal_var=False)
    ttest_stat, ttest_p = ttest_res.statistic, ttest_res.pvalue
else:
    ttest_stat, ttest_p = np.nan, np.nan

# Chi2: sentiment vs win (use top two sentiments if many)
try:
    chi2_res = stats.chi2_contingency(pd.crosstab(merged['classification'], merged['win']))
except Exception as e:
    chi2_res = None

# Simple model: predict win using [is_long, leverage, size, sentiment_onehots]
model_info = {}
model_df = merged[merged['classification'].isin(['Fear','Greed'])].copy()
if len(model_df) >= 50:
    model_df['is_long'] = (model_df['side']=='buy').astype(int)
    X = model_df[['is_long','leverage','size']].fillna(0)
    X = pd.concat([X, pd.get_dummies(model_df['classification'], prefix='sent')], axis=1)
    y = model_df['win']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)
    clf = LogisticRegression(max_iter=2000)
    clf.fit(X_train_s, y_train)
    y_pred = clf.predict(X_test_s)
    y_proba = clf.predict_proba(X_test_s)[:,1]
    report = classification_report(y_test, y_pred, output_dict=True)
    roc = roc_auc_score(y_test, y_proba) if len(np.unique(y_test))>1 else np.nan
    coefs = pd.DataFrame({'feature': X.columns, 'coef': clf.coef_[0]})
    model_info = {
        'classification_report': report,
        'roc_auc': roc,
        'coefficients': coefs
    }
else:
    print("Not enough data for predictive model (need >=50 rows with Fear/Greed).")

# -----------------------------
# 8) Build HTML report with embedded images (base64) + tables
# -----------------------------
def df_to_html_table(df, title=None):
    html = ""
    if title:
        html += f"<h3>{title}</h3>\n"
    html += df.to_html(index=False, classes='table', border=0)
    return html

html_parts = []
html_parts.append("<html><head><meta charset='utf-8'><title>Trader vs Sentiment Analysis</title>")
html_parts.append("""<style>
body { font-family: Arial, Helvetica, sans-serif; margin: 30px; color: #222; }
h1 { color: #0B5FFF; }
.table { border-collapse: collapse; width: 100%; margin-bottom: 20px;}
.table th, .table td { border: 1px solid #ddd; padding: 8px; }
.section { margin-bottom: 30px; }
.small { font-size: 0.9em; color: #555; }
</style>""")
html_parts.append("</head><body>")
html_parts.append(f"<h1>Trader vs Market Sentiment Analysis</h1>")
html_parts.append(f"<p class='small'>Generated: {datetime.utcnow().isoformat()} UTC</p>")

# Overview
html_parts.append("<div class='section'><h2>Overview</h2>")
html_parts.append(f"<p>Number of trades analyzed: <strong>{len(merged):,}</strong></p>")
html_parts.append("<p>Sentiment distribution:</p>")
html_parts.append("<pre>" + merged['classification'].value_counts().to_string() + "</pre>")
html_parts.append("</div>")

# Plots
html_parts.append("<div class='section'><h2>Plots</h2>")
html_parts.append("<h3>PnL distribution by sentiment</h3>")
html_parts.append(f"<img src='data:image/png;base64,{b64_1}' alt='PnL by Sentiment' style='max-width:100%;height:auto;border:1px solid #ccc;padding:4px;'/>")
html_parts.append("<h3>Win Rate by Sentiment</h3>")
html_parts.append(f"<img src='data:image/png;base64,{b64_2}' alt='Win Rate' style='max-width:100%;height:auto;border:1px solid #ccc;padding:4px;'/>")
html_parts.append("<h3>Mean PnL by Leverage Bucket and Sentiment</h3>")
html_parts.append(f"<img src='data:image/png;base64,{b64_3}' alt='Leverage vs PnL' style='max-width:100%;height:auto;border:1px solid #ccc;padding:4px;'/>")
html_parts.append("<h3>7-day rolling total PnL</h3>")
html_parts.append(f"<img src='data:image/png;base64,{b64_4}' alt='Rolling PnL' style='max-width:100%;height:auto;border:1px solid #ccc;padding:4px;'/>")
if plotly_html_path:
    html_parts.append(f"<p>Interactive Plotly chart saved separately: {os.path.basename(plotly_html_path)}</p>")
html_parts.append("</div>")

# Tables & stats
html_parts.append("<div class='section'><h2>Aggregates & Statistics</h2>")
html_parts.append(df_to_html_table(sent_agg, title="Per-Sentiment Aggregates"))
if not acct_agg.empty:
    html_parts.append(df_to_html_table(acct_agg.sort_values('total_pnl', ascending=False).head(50), title="Top 50 accounts by total PnL"))
html_parts.append("<h3>Statistical tests</h3>")
html_parts.append("<ul>")
html_parts.append(f"<li>T-test (Fear vs Greed) on PnL: statistic={ttest_stat}, p-value={ttest_p}</li>")
if chi2_res is not None:
    html_parts.append(f"<li>Chi2 contingency (classification vs win): chi2_stat={chi2_res[0]:.4f}, p={chi2_res[1]:.4g}</li>")
else:
    html_parts.append("<li>Chi2 contingency could not be computed (insufficient data).</li>")
html_parts.append("</ul>")
html_parts.append("</div>")

# Model summary if available
if model_info:
    html_parts.append("<div class='section'><h2>Predictive model (Logistic Regression)</h2>")
    html_parts.append(f"<p>ROC AUC: {model_info['roc_auc']:.4f}</p>")
    # coefficients table
    html_parts.append(df_to_html_table(model_info['coefficients'].sort_values('coef', key=abs, ascending=False), title="Model coefficients"))
    html_parts.append("</div>")

# Footer
html_parts.append("<div class='section small'><p>Notes: This report was generated automatically. Check the saved CSV files for raw+aggregated outputs.</p></div>")

html_parts.append("</body></html>")
report_html = "\n".join(html_parts)

# Save HTML
html_path = os.path.join(out_dir, 'analysis_report.html')
with open(html_path, 'w', encoding='utf-8') as f:
    f.write(report_html)
print("Saved HTML report to:", html_path)

# -----------------------------
# 9) Convert HTML -> PDF (WeasyPrint if available, else ReportLab fallback)
# -----------------------------
pdf_path = os.path.join(out_dir, 'analysis_report.pdf')
if use_weasy:
    try:
        HTML(string=report_html).write_pdf(pdf_path)
        print("Saved PDF report via WeasyPrint to:", pdf_path)
    except Exception as e:
        print("WeasyPrint failed to generate PDF, falling back to ReportLab. Error:", e)
        use_weasy = False

if not use_weasy:
    # Fallback: create a multi-page PDF using ReportLab by laying out the HTML contents and images.
    # We'll place text summary + each saved PNG on its own page.
    try:
        c = canvas.Canvas(pdf_path, pagesize=letter)
        width, height = letter
        # Title page
        c.setFont("Helvetica-Bold", 18)
        c.drawString(40, height-80, "Trader vs Market Sentiment Analysis")
        c.setFont("Helvetica", 10)
        c.drawString(40, height-100, f"Generated: {datetime.utcnow().isoformat()} UTC")
        c.drawString(40, height-120, f"Number of trades analyzed: {len(merged):,}")
        c.showPage()
        # Add Sentiment table as text (first 80 rows)
        c.setFont("Helvetica-Bold", 14)
        c.drawString(40, height-40, "Per-Sentiment Aggregates (top rows):")
        c.setFont("Helvetica", 9)
        text = c.beginText(40, height-60)
        sent_str = sent_agg.head(80).to_string(index=False)
        for line in sent_str.splitlines():
            text.textLine(line)
        c.drawText(text)
        c.showPage()
        # Add each PNG as a full-page image
        for imgfile in [path1, path2, path3, path4]:
            try:
                img = Image.open(imgfile)
                # preserve aspect ratio fit to page with margins
                maxw = width - 80
                maxh = height - 120
                img_w, img_h = img.size
                ratio = min(maxw/img_w, maxh/img_h)
                draw_w = img_w * ratio
                draw_h = img_h * ratio
                # save to temp file as JPEG to ensure compatibility
                tmp_path = imgfile + ".tmp.jpg"
                img.convert('RGB').save(tmp_path, "JPEG")
                c.drawImage(tmp_path, 40, (height - 60 - draw_h), width=draw_w, height=draw_h)
                c.showPage()
                os.remove(tmp_path)
            except Exception as e:
                print("Failed to add image to PDF fallback:", e)
        c.save()
        print("Saved PDF report via ReportLab fallback to:", pdf_path)
    except Exception as e:
        print("ReportLab fallback failed. Error:", e)
        print("At least the HTML report was saved at:", html_path)

# -----------------------------
# 10) Save CSV outputs, model artifacts, and Streamlit app script
# -----------------------------
merged_path = os.path.join(out_dir, 'merged_trades_sentiment.csv')
merged.to_csv(merged_path, index=False)
sent_agg.to_csv(os.path.join(out_dir, 'sentiment_aggregates.csv'), index=False)
acct_path = os.path.join(out_dir, 'account_aggregates.csv')
if not acct_agg.empty:
    acct_agg.to_csv(acct_path, index=False)
if model_info:
    model_info['coefficients'].to_csv(os.path.join(out_dir, 'model_coefficients.csv'), index=False)

print("Saved merged CSV to:", merged_path)
if not acct_agg.empty:
    print("Saved account aggregates to:", acct_path)

# Streamlit app (simple)
streamlit_code = f"""# Streamlit dashboard for Trader vs Sentiment
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title='Trader vs Sentiment Dashboard', layout='wide')
st.title('Trader vs Sentiment Dashboard')

merged = pd.read_csv('{merged_path}')

st.sidebar.header('Filters')
sent_filter = st.sidebar.multiselect('Sentiment', merged['classification'].unique().tolist(), default=merged['classification'].unique().tolist())

df = merged[merged['classification'].isin(sent_filter)]

st.header('Summary')
st.metric('Trades', f\"{{len(df):,}}\")
st.metric('Avg PnL', f\"{{df['pnl'].mean():.4f}}\")

st.header('PnL by Sentiment (boxplot)')
fig = px.box(df[df['classification']!='Unknown'], x='classification', y='pnl', points='outliers')
st.plotly_chart(fig, use_container_width=True)

st.header('Leverage distribution by Sentiment')
fig2 = px.histogram(df, x='leverage', color='classification', nbins=30)
st.plotly_chart(fig2, use_container_width=True)

st.header('Per-account table (top 50 by total PnL)')
if 'account' in df.columns:
    acct = df.groupby('account').agg(trades=('pnl','count'), total_pnl=('pnl','sum'), win_rate=('win','mean')).reset_index().sort_values('total_pnl', ascending=False).head(50)
    st.dataframe(acct)
else:
    st.info('No account column present in dataset.')
"""

streamlit_file = os.path.join(out_dir, 'dashboard_app.py')
with open(streamlit_file, 'w') as f:
    f.write(streamlit_code)
print("Saved Streamlit app script to:", streamlit_file)
print("To run the dashboard locally or in Colab (if supported), run:")
print(f"  streamlit run {streamlit_file}")

# -----------------------------
# 11) Final message & summary
# -----------------------------
print("\n=== Finished ===")
print("Outputs saved to directory:", out_dir)
print("Key files:")
print(" - HTML report:", html_path)
print(" - PDF report:", pdf_path)
print(" - Merged CSV:", merged_path)
print(" - Sentiment aggregates CSV:", os.path.join(out_dir, 'sentiment_aggregates.csv'))
print(" - Streamlit app:", streamlit_file)
if plotly_html_path:
    print(" - Interactive Plotly HTML:", plotly_html_path)
print("\nOpen the HTML report to inspect visuals immediately. If PDF conversion fails, you still have the HTML and PNG images.")

Installing required packages (this may take a minute)...
Loading /content/historical_data.csv and /content/fear_greed_index.csv
Trades shape: (211224, 16)
Sentiment shape: (2644, 4)
Parsed trades time from column: time
Merged shape: (0, 20)
Sentiment distribution in merged:
Series([], Name: count, dtype: int64)


DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.002s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.005s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
INFO:fontTools.subset:kern dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:f

Not enough data for predictive model (need >=50 rows with Fear/Greed).
Saved HTML report to: /content/trader_sentiment_results/analysis_report.html


INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
INFO:fontTools.subset:kern dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:fontTools.ttLib.ttFont:Reading 'glyf' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'glyf' table
DEBUG:fontTools.ttLib.ttFont:Reading 'loca' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'loca' table
DEBUG:fontTools.ttLib.ttFont:Reading 'head' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'head' table
DEBUG:fontTools.subset.timer:Took 0.005s to load 'glyf'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'glyf'
INFO:fontTools.subset:glyf pruned
DEBUG:fontTools.subset.timer:Took 0.000s to close glyph list over 'cmap'
INFO:fontTools.subset:Added gid0 to subset
INFO:fontTools.subset:Closing glyph list over 'glyf': 55 glyphs before
IN

Saved PDF report via WeasyPrint to: /content/trader_sentiment_results/analysis_report.pdf
Saved merged CSV to: /content/trader_sentiment_results/merged_trades_sentiment.csv
Saved Streamlit app script to: /content/trader_sentiment_results/dashboard_app.py
To run the dashboard locally or in Colab (if supported), run:
  streamlit run /content/trader_sentiment_results/dashboard_app.py

=== Finished ===
Outputs saved to directory: /content/trader_sentiment_results
Key files:
 - HTML report: /content/trader_sentiment_results/analysis_report.html
 - PDF report: /content/trader_sentiment_results/analysis_report.pdf
 - Merged CSV: /content/trader_sentiment_results/merged_trades_sentiment.csv
 - Sentiment aggregates CSV: /content/trader_sentiment_results/sentiment_aggregates.csv
 - Streamlit app: /content/trader_sentiment_results/dashboard_app.py
 - Interactive Plotly HTML: /content/trader_sentiment_results/plotly_pnl_by_sentiment.html

Open the HTML report to inspect visuals immediately. If PDF